In [3]:
import sys
sys.path.append('..')
import H2_match_coeffs as gmc
import utils.input_output as io
import numpy as np
import torch
import enr.H2 as energy
use_cuda = 1
torchdeviceId = torch.device('cuda:0') if use_cuda else 'cpu'
torchdtype = torch.float32
import os


In [4]:
path_dfaust_template = sys.path[0]+"/base/DFAUST_template.ply"
[v_template,faces,Fun0] = io.loadData(path_dfaust_template) 
v_template = torch.from_numpy(v_template).to(dtype=torchdtype, device=torchdeviceId)
faces = torch.from_numpy(faces).to(dtype=torch.long, device=torchdeviceId)

In [7]:
basis_np_mot = np.load(sys.path[0]+"/base/basis_DFAUST_motions.npy")
b_split= basis_np_mot.shape[0]
basis_np_shape = np.load(sys.path[0]+"/base/basis_DFAUST_shape.npy")
basis_np = np.concatenate((basis_np_mot,basis_np_shape),axis=0)

basis_torch_shape = torch.from_numpy(basis_np_shape).to(dtype=torchdtype, device=torchdeviceId)
basis_torch_mot = torch.from_numpy(basis_np_mot).to(dtype=torchdtype, device=torchdeviceId)
basis_torch = torch.from_numpy(basis_np).to(dtype=torchdtype, device=torchdeviceId)

print(b_split, basis_np.shape)

130 (170, 6890, 3)


In [8]:
a0=1
a1=1000
b1=100
c1=1
d1=1
a2=1

param1 = { 'weight_coef_dist': 10**1,'sig_geom':.4,'kernel_grass': 'linear',
          'max_iter': 500,'time_steps': 2}
param2 = { 'weight_coef_dist': 10**6,'sig_geom':.1,'kernel_grass': 'linear',
          'max_iter': 500,'time_steps': 2}
param3 = { 'weight_coef_dist': 10**12,'sig_geom':.025,'kernel_grass': 'linear',
          'max_iter': 1000,'time_steps': 2}

paramlist=[param1,param2,param3]

In [16]:
path_to_target = ""
[V_raw,F_raw,FunS] = io.loadData(path_to_target)
target = [V_raw,F_raw]

init_path = torch.stack([v_template]*5,dim=0)


chemin_exp,X = gmc.H2MultiRes_coeff(a0, a1, b1, c1, d1, a2, paramlist, [v_template.cpu().numpy(), faces.cpu().numpy()], target, init_path, faces, basis_torch)
chemin_exp =chemin + torch.einsum("ij, jkl-> ikl",X, basis_torch)     
io.plotGeodesic([chemin_exp[-1].cpu().numpy()],faces.cpu().numpy(),stepsize=2,axis=[0,1,0],angle=2*np.pi)
